In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/processed-uatd/processed_uatd/processed_test_2/annotations/00270.xml
/kaggle/input/processed-uatd/processed_uatd/processed_test_2/annotations/00793.xml
/kaggle/input/processed-uatd/processed_uatd/processed_test_2/annotations/00064.xml
/kaggle/input/processed-uatd/processed_uatd/processed_test_2/annotations/00106.xml
/kaggle/input/processed-uatd/processed_uatd/processed_test_2/annotations/00418.xml
/kaggle/input/processed-uatd/processed_uatd/processed_test_2/annotations/00249.xml
/kaggle/input/processed-uatd/processed_uatd/processed_test_2/annotations/00634.xml
/kaggle/input/processed-uatd/processed_uatd/processed_test_2/annotations/00259.xml
/kaggle/input/processed-uatd/processed_uatd/processed_test_2/annotations/00061.xml
/kaggle/input/processed-uatd/processed_uatd/processed_test_2/annotations/00262.xml
/kaggle/input/processed-uatd/processed_uatd/processed_test_2/annotations/00222.xml
/kaggle/input/processed-uatd/processed_uatd/processed_test_2/annotations/00307.xml
/kag

In [2]:
import os
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report
import numpy as np

In [3]:
train_folder = "/kaggle/input/processed-uatd/processed_uatd/processed_training"
test_1_folder = "/kaggle/input/processed-uatd/processed_uatd/processed_test_1"
test_2_folder = "/kaggle/input/processed-uatd/processed_uatd/processed_test_2"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import xml.etree.ElementTree as ET
import os
import numpy as np
from sklearn.metrics import classification_report

# Manually define the list of class names
classes = [
    'cube', 
    'cylinder', 
    'tyres', 
    'ball', 
    'human body', 
    'circle cage', 
    'square cage', 
    'metal bucket', 
    'plane model', 
    'rov'
]

# Normalization map to handle alternate labels
normalization_map = {
    'tyre': 'tyres',
    'tire': 'tyres',
    'human': 'human body',
    'body': 'human body',
    'rov': 'rov',
    'robot': 'rov',
    'bucket': 'metal bucket',
    'plane': 'plane model',
    'ball': 'ball',
    'cube': 'cube',
    'cylinder': 'cylinder',
    'circle cage': 'circle cage',
    'square cage': 'square cage'
}

class CustomImageAnnotationDataset(Dataset):
    def __init__(self, images_dir, annotations_dir, transform=None, class_names=None):
        self.images_dir = images_dir
        self.annotations_dir = annotations_dir
        self.transform = transform
        self.class_names = class_names
        self.class_to_idx = {name: idx for idx, name in enumerate(class_names)}
        self.image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
        self.image_files.sort()

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.images_dir, img_name)
        xml_name = img_name.replace('.jpg', '.xml')
        xml_path = os.path.join(self.annotations_dir, xml_name)

        image = Image.open(img_path).convert('RGB')
        labels = self.parse_annotation(xml_path)

        if self.transform:
            image = self.transform(image)

        # Assume single label per image
        label = labels[0] if labels else -1  # or use a default value
        return image, label

    def parse_annotation(self, xml_path):
        tree = ET.parse(xml_path)
        root = tree.getroot()
        labels = []

        for obj in root.findall('object'):
            raw_label = obj.find('name').text.lower().strip()
            normalized_label = normalization_map.get(raw_label, raw_label)

            if normalized_label in self.class_to_idx:
                labels.append(self.class_to_idx[normalized_label])
            else:
                print(f"[WARNING] Unknown class '{raw_label}' (normalized as '{normalized_label}') in {xml_path}")

        return labels

# Define transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Initialize the dataset
full_dataset = CustomImageAnnotationDataset(
    images_dir='/kaggle/input/processed-uatd/processed_uatd/processed_training/images',
    annotations_dir='/kaggle/input/processed-uatd/processed_uatd/processed_training/annotations',
    transform=transform,
    class_names=classes
)

# Split the dataset
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_data, test_data = random_split(full_dataset, [train_size, test_size])

# Data loaders
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Model setup
model = models.resnet18(weights='IMAGENET1K_V1')
model.fc = nn.Linear(model.fc.in_features, len(classes))

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

# Evaluation
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

print("\n=== Classification Report ===")
print(classification_report(all_labels, all_preds, target_names=classes))


Epoch 1/5, Loss: 0.6723
Epoch 2/5, Loss: 0.4472
Epoch 3/5, Loss: 0.3503
Epoch 4/5, Loss: 0.3069
Epoch 5/5, Loss: 0.2520

=== Classification Report ===
              precision    recall  f1-score   support

        cube       0.81      0.89      0.85       403
    cylinder       0.91      0.60      0.72        80
       tyres       0.76      0.83      0.79       138
        ball       0.69      0.82      0.75       273
  human body       0.92      0.92      0.92       200
 circle cage       0.63      0.63      0.63        46
 square cage       0.82      0.54      0.65        57
metal bucket       0.67      0.15      0.25        13
 plane model       1.00      0.70      0.82       165
         rov       0.99      1.00      1.00       145

    accuracy                           0.82      1520
   macro avg       0.82      0.71      0.74      1520
weighted avg       0.84      0.82      0.82      1520



# Testing

In [5]:
from PIL import Image
import torch
from torchvision import transforms

# Ensure model is on correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load a test image
image_path = "/kaggle/input/processed-uatd/processed_uatd/processed_test_1/images/00001.jpg"
image = Image.open(image_path).convert("RGB")

# Define the same transform used during training (with normalization)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Match what ResNet expects
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Apply transform and add batch dimension
input_tensor = transform(image).unsqueeze(0).to(device)

# Run inference
model.eval()
with torch.no_grad():
    output = model(input_tensor)
    _, predicted_class = torch.max(output, 1)

# Get label name from class index
predicted_label = full_dataset.class_names[predicted_class.item()]
print("Predicted Class:", predicted_label)


Predicted Class: rov
